In [2]:
import pandas as pd
import re
import string
from nltk.corpus import stopwords
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from numpy import asarray
from numpy import zeros
from keras.utils import to_categorical

In [3]:
data_train=pd.read_csv('train.csv',header=None)[0:100]
data_train.columns=['class','title','content']
data_test=pd.read_csv('test.csv',header=None)[0:100]
data_test.columns=['class','title','content']

In [4]:
data_train

,class,title,content
0,2,Wizards Guard Blake Out 10-12 Week (AP),AP - Washington Wizards point guard Steve Blak...
1,4,Brief: HP acquires Synstar for \$293M,In a move designed to help it better compete i...
2,3,US Airways #39; pilots vote for 18 salary redu...,US Airways #39; pilots voted to approve a new ...
3,4,Radiation risks 'need updating',A UK panel examining radiation risks says offi...
4,1,President's fate is on the line in Venezuela,CARACAS -- Partisans on both sides are calling...
5,3,Australia #39;s Virgin Blue increases fuel sur...,SYDNEY : Australian airline Virgin Blue announ...
6,1,Chinese sub detected in Japan waters,Japanese government officials Thursday confirm...
7,1,From War-Torn to Wonderland,Honduras and other Central American countries ...
8,2,Japan celebrates Suzuki breaking record,Baseball fans across Japan Saturday honored Ic...
9,2,"On world stage, no one else compares to Els",It was two years ago when Ernie Els explained ...


In [5]:
content_train = []
for i in range(0,len(data_train['title'])):
    A = data_train['content'][i].split()
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # remove punctuation from each word
    A = [re_punc.sub('', w) for w in A]
    # remove non-printatble characters
    re_print = re.compile('[^%s]'%re.escape(string.printable))
    A = [re_print.sub('', w) for w in A]
    # convert to lower case
    A = [word.lower() for word in A]
    # remove remaining tokens that are not alphabetic
    A = [word for word in A if word.isalpha()]
    # filter out stopwords
    stop_words = set(stopwords.words('english'))
    A = [w for w in A if not w in stop_words]
    # filter out short tokens
    A = [word for word in A if len(word) > 1]
    A = " ".join(A)
    content_train.append(A)

In [6]:
content_test = []
for i in range(0,len(data_test['title'])):
    A = data_test['content'][i].split()
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # remove punctuation from each word
    A = [re_punc.sub('', w) for w in A]
    # remove non-printatble characters
    re_print = re.compile('[^%s]'%re.escape(string.printable))
    A = [re_print.sub('', w) for w in A]
    # convert to lower case
    A = [word.lower() for word in A]
    # remove remaining tokens that are not alphabetic
    A = [word for word in A if word.isalpha()]
    # filter out stopwords
    stop_words = set(stopwords.words('english'))
    A = [w for w in A if not w in stop_words]
    # filter out short tokens
    A = [word for word in A if len(word) > 1]
    A = " ".join(A)
    content_test.append(A)

In [7]:
title_train = []
for i in range(0,len(data_train['title'])):
    title_train.append(data_train['title'][i])

title_test = []
for i in range(0,len(data_test['title'])):
    title_test.append(data_test['title'][i])   

In [8]:
Class_train = data_train['class']
    
Class_test = data_test['class']

In [8]:
type(Class_train)

pandas.core.series.Series

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab
vectorizer.fit(content_train)
# summarize
print(vectorizer.vocabulary_)
print(vectorizer.idf_)
# encode document
vector_train = vectorizer.transform(content_train)
vector_test = vectorizer.transform(content_test)
# summarize encoded vector
print(vector_train.shape)
print(vector_train.toarray())

{'ap': 61, 'washington': 1302, 'wizards': 1330, 'point': 877, 'guard': 502, 'steve': 1145, 'blake': 131, 'miss': 740, 'first': 434, 'month': 750, 'season': 1052, 'injuring': 588, 'ankle': 56, 'pickup': 861, 'game': 468, 'move': 755, 'designed': 323, 'help': 522, 'better': 124, 'compete': 240, 'europe': 394, 'hewlettpackard': 525, 'said': 1037, 'acquired': 9, 'synstar': 1179, 'plc': 874, 'ukbased': 1259, 'technology': 1195, 'services': 1074, 'firm': 433, 'million': 734, 'us': 1274, 'airways': 38, 'pilots': 863, 'voted': 1292, 'approve': 71, 'new': 773, 'labor': 637, 'agreement': 28, 'yesterday': 1348, 'reduce': 975, 'salaries': 1038, 'percent': 849, 'save': 1043, 'airline': 35, 'year': 1345, 'uk': 1258, 'panel': 832, 'examining': 400, 'radiation': 945, 'risks': 1017, 'says': 1045, 'official': 796, 'estimates': 392, 'dangers': 294, 'health': 516, 'may': 708, 'wide': 1321, 'mark': 699, 'substantially': 1159, 'caracas': 173, 'partisans': 836, 'sides': 1087, 'calling': 166, 'polarized': 878

In [10]:
# from keras.preprocessing.text import Tokenizer
# fit a tokenizer
# def create_tokenizer(lines):
#     tokenizer = Tokenizer()
#     tokenizer.fit_on_texts(lines)
#     return tokenizer
# create the tokenizer
# tokenizer = create_tokenizer(content_train)
# encode data
# Xtrain = tokenizer.texts_to_matrix(content_train, mode='tfidf')
# Xtest = tokenizer.texts_to_matrix(content_test, mode='tfidf')

In [11]:
rf = RandomForestClassifier(n_estimators = 500, max_depth = 3, random_state = 123, n_jobs = -1)
rf.fit(vector_train, Class_train)
rfpreds = rf.predict(vector_test)
accuracy_score(Class_test,rfpreds)

0.32

Word2Vec

In [10]:
from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import *
# create the tokenizer
tokenizer = Tokenizer()
# fit the tokenizer on the documents

tokenizer.fit_on_texts(content_train) 
# sequence encode
encoded_train = tokenizer.texts_to_sequences(content_train)
# pad sequences
max_length= max([len(s.split()) for s in content_train])
Xtrain = pad_sequences(encoded_train, maxlen=max_length, padding='post')
# define training labels
ytrain = to_categorical(Class_train)
# Xtrain.shape
# type(Xtrain)
Xtrain[0]

array([  5,  58, 301, 302, 114, 303, 304, 305,  33, 306,  22, 307, 308,
       309,  59,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0], dtype=int32)

In [13]:
encoded_test = tokenizer.texts_to_sequences(content_test)
Xtest = pad_sequences(encoded_test, maxlen=max_length, padding='post')
ytest = to_categorical(Class_test)

In [14]:
content_train_wv = []
for i in range(0,len(data_train['title'])):
    A = data_train['content'][i].split()
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # remove punctuation from each word
    A = [re_punc.sub('', w) for w in A]
    # remove non-printatble characters
    re_print = re.compile('[^%s]'%re.escape(string.printable))
    A = [re_print.sub('', w) for w in A]
    # convert to lower case
    A = [word.lower() for word in A]
    # remove remaining tokens that are not alphabetic
    A = [word for word in A if word.isalpha()]
    # filter out stopwords
    stop_words = set(stopwords.words('english'))
    A = [w for w in A if not w in stop_words]
    # filter out short tokens
    A = [word for word in A if len(word) > 1]
    content_train_wv.append(A)


In [15]:
from gensim.models import Word2Vec
# train word2vec model
model_train = Word2Vec(content_train_wv, size=100, window=2, workers=4, min_count=2)

unable to import 'smart_open.gcs', disabling that module


In [16]:
# summarize vocabulary size in model
words = list(model_train.wv.vocab)
print('Vocabulary size: %d' % len(words))

Vocabulary size: 300


In [17]:
# save model in ASCII (word2vec) format
filename = 'embedding_word2vec.txt'
model_train.wv.save_word2vec_format(filename, binary=False)

In [18]:
# load embedding as a dict
def load_embedding(filename):
# load embedding into memory, skip first line
    file = open(filename,'r')
    lines = file.readlines()[1:]
    file.close()
    # create a map of words to vectors
    embedding = dict()
    for line in lines:
        parts = line.split()
    # key is string word, value is numpy array for vector
        embedding[parts[0]] = asarray(parts[1:], dtype='float32')
    return embedding

In [19]:
# create a weight matrix for the Embedding layer from a loaded embedding
def get_weight_matrix(embedding, vocab):
# total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = zeros((vocab_size, 100))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        weight_matrix[i] = embedding.get(word)
    return weight_matrix

In [35]:
# define vocabulary size (largest integer value)
vocab_size = len(tokenizer.word_index) + 1
# load embedding from file
raw_embedding = load_embedding('embedding_word2vec.txt')
# get vectors in the right order
embedding_vectors = get_weight_matrix(raw_embedding, tokenizer.word_index)
# create the embedding layer
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_vectors], input_length=max_length, trainable=False)
# embedding_layer.get_weights()

In [45]:
# define model
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(5, activation='softmax'))
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
train = np.asarray(content_train_wv)
train.shape
# model.fit(content_train_wv, ytrain, epochs=10, verbose=2)
# evaluate
# loss, acc = model.evaluate(Xtest, ytest, verbose=0)

# print('Test Accuracy: %f' % (acc*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 42, 100)           135300    
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 38, 128)           64128     
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 19, 128)           0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 2432)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 5)                 12165     
Total params: 211,593
Trainable params: 76,293
Non-trainable params: 135,300
_________________________________________________________________
None


(100,)